## 数据处理

In [15]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from collections import Counter
import torch
from sklearn.model_selection import train_test_split

# 数据读取
data = pd.read_csv('C:/Users/ASUS/BERT/AI_Human.csv')

# 数据采样与清洗
ai_samples = data[data['generated'] == 1]
human_samples = data[data['generated'] == 0]
data = pd.concat([ai_samples.sample(n=5000, random_state=42), human_samples.sample(n=5000, random_state=42)])
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# 清洗函数
def remove_punc(text):
    return ''.join([char for char in text if char not in punctuation])

def remove_stop(text):
    stops = set(stopwords.words('english'))
    return " ".join([word for word in text.split() if word.lower() not in stops])

# 文本清洗
data['cleaned'] = data['text'].str.lower()
data['cleaned'] = data['cleaned'].apply(lambda x: re.sub(r'https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE))
data['cleaned'] = data['cleaned'].apply(lambda x: re.sub(r'<.*?>', '', x))
data['cleaned'] = data['cleaned'].apply(remove_punc)
data['cleaned'] = data['cleaned'].apply(remove_stop)
data['tokens'] = data['cleaned'].apply(word_tokenize)
data.rename(columns={'generated': 'label'}, inplace=True)
data



,text,label,cleaned,tokens
0,"Dear Principal,\n\nWe have been hearing quite ...",0.0,dear principal hearing quite lot subject commu...,"[dear, principal, hearing, quite, lot, subject..."
1,"Dear [State Senator],\n\nI am writing to expr...",1.0,dear state senator writing express opinion ele...,"[dear, state, senator, writing, express, opini..."
2,"As high school students, we are constantly bom...",1.0,high school students constantly bombarded info...,"[high, school, students, constantly, bombarded..."
3,"Hi, I'm a 6th garden and I think zoos ANE Near...",1.0,hi im 6th garden think zoos ane nearly cool iv...,"[hi, im, 6th, garden, think, zoos, ane, nearly..."
4,"Sure, jar's my attempt at writing an essay as ...",1.0,sure jars attempt writing essay average 8tj gr...,"[sure, jars, attempt, writing, essay, average,..."
...,...,...,...,...
9995,Good actions can be helpful in most ways. Good...,0.0,good actions helpful ways good altered led goo...,"[good, actions, helpful, ways, good, altered, ..."
9996,"The article ""Unmaking the Face on Mars"" explai...",0.0,article unmaking face mars explains face mars ...,"[article, unmaking, face, mars, explains, face..."
9997,Driving can be extremely dangerous for you or ...,0.0,driving extremely dangerous anyone else especi...,"[driving, extremely, dangerous, anyone, else, ..."
9998,"Hey there! \n\nSo, you know how people say ""ki...",1.0,hey know people say kindness goes long way yea...,"[hey, know, people, say, kindness, goes, long,..."


In [17]:
# 创建词汇表
vocab = Counter(word for tokens in data['tokens'] for word in tokens)
vocab_size = len(vocab)

# 创建词到索引的映射
word_to_idx = {word: idx + 1 for idx, (word, _) in enumerate(vocab.most_common())}

# 将文本转化为数字序列
data['numerical'] = data['tokens'].apply(lambda x: [word_to_idx[word] for word in x if word in word_to_idx])

# # 填充序列
# max_len = 200
# def truncate_and_pad(sequences, max_len):
#     padded_sequences = []
#     for seq in sequences:
#         if len(seq) > max_len:  # 裁剪
#             padded_sequences.append(seq[:max_len])
#         else:  # 填充
#             padded_sequences.append(seq + [0] * (max_len - len(seq)))
#     return padded_sequences

# X = truncate_and_pad(data['numerical'].values, max_len)
# y = data['label'].values

# # 数据划分
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=100)

# # 转为 PyTorch 张量
# X_train_tensor = torch.tensor(X_train, dtype=torch.long)
# X_test_tensor = torch.tensor(X_test, dtype=torch.long)
# y_train_tensor = torch.tensor(y_train, dtype=torch.long)
# y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
# test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# # 保存数据处理后的数据
# torch.save({
#     'train_loader': train_loader,
#     'test_loader': test_loader,
#     'vocab_size': vocab_size,
#     'word_to_idx': word_to_idx,  # 保存词到索引的映射
# }, './processed_data.pt')
vocab_size

40157

In [18]:
# 数据读取
data = pd.read_csv('C:/Users/ASUS/BERT/data_Enhance/enhanceded_ai_human.csv')

# 数据采样与清洗
ai_samples = data[data['label'] == 1]
human_samples = data[data['label'] == 0]
data = pd.concat([ai_samples.sample(n=2000, random_state=42), human_samples.sample(n=2000, random_state=42)])
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# 清洗函数
def remove_punc(text):
    return ''.join([char for char in text if char not in punctuation])

def remove_stop(text):
    stops = set(stopwords.words('english'))
    return " ".join([word for word in text.split() if word.lower() not in stops])

# 文本清洗
data['cleaned'] = data['text'].str.lower()
data['cleaned'] = data['cleaned'].apply(lambda x: re.sub(r'https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE))
data['cleaned'] = data['cleaned'].apply(lambda x: re.sub(r'<.*?>', '', x))
data['cleaned'] = data['cleaned'].apply(remove_punc)
data['cleaned'] = data['cleaned'].apply(remove_stop)
data['tokens'] = data['cleaned'].apply(word_tokenize)

data_aug=data
data_aug

,text,label,cleaned,tokens
0,Certainly! Below is a reworded version of your...,1,certainly reworded version passage repetitive ...,"[certainly, reworded, version, passage, repeti..."
1,"About twenty-five years ago, something happene...",0,twentyfive years ago something happened around...,"[twentyfive, years, ago, something, happened, ..."
2,Certainly! Here’s a reworded version with repe...,1,certainly here’s reworded version repetitive p...,"[certainly, here, ’, s, reworded, version, rep..."
3,Dt is important for students to continue learn...,0,dt important students continue learning summer...,"[dt, important, students, continue, learning, ..."
4,"In ""The Challenge of Exploring Venus"" the auth...",0,challenge exploring venus author suggests stud...,"[challenge, exploring, venus, author, suggests..."
...,...,...,...,...
3995,Certainly! Below is your passage reworded with...,1,certainly passage reworded repetitive phrases ...,"[certainly, passage, reworded, repetitive, phr..."
3996,**A Good Attitude: The Key to Success** \n\nA...,1,good attitude key success good attitude vital ...,"[good, attitude, key, success, good, attitude,..."
3997,Certainly! Here’s a reworded version with repe...,1,certainly here’s reworded version repetitive p...,"[certainly, here, ’, s, reworded, version, rep..."
3998,Do You Know What The Electoral College Is?\n\n...,0,know electoral college electoral college place...,"[know, electoral, college, electoral, college,..."


In [19]:
# 创建词汇表
vocab = Counter(word for tokens in data_aug['tokens'] for word in tokens)
vocab_size = len(vocab)

# 创建词到索引的映射
word_to_idx = {word: idx + 1 for idx, (word, _) in enumerate(vocab.most_common())}

# 将文本转化为数字序列
data_aug['numerical'] = data_aug['tokens'].apply(lambda x: [word_to_idx[word] for word in x if word in word_to_idx])

# # 填充序列
# max_len = 200
# def truncate_and_pad(sequences, max_len):
#     padded_sequences = []
#     for seq in sequences:
#         if len(seq) > max_len:  # 裁剪
#             padded_sequences.append(seq[:max_len])
#         else:  # 填充
#             padded_sequences.append(seq + [0] * (max_len - len(seq)))
#     return padded_sequences

# X = truncate_and_pad(data_aug['numerical'].values, max_len)
# y = data_aug['label'].values

# # 数据划分
# X_train, _, y_train, _ = train_test_split(X, y, test_size=0.1, random_state=100)

# # 转为 PyTorch 张量
# X_train_tensor = torch.tensor(X_train, dtype=torch.long)
# y_train_tensor = torch.tensor(y_train, dtype=torch.long)

# train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# # 保存数据处理后的数据
# torch.save({
#     'train_loader_aug': train_loader,
#     'test_loader_aug': test_loader,
#     'vocab_size_aug': vocab_size,
#     'word_to_idx_aug': word_to_idx,  # 保存词到索引的映射
# }, './processed_data_aug.pt')
vocab_size

26503

In [22]:
from collections import Counter

# 假设 data_original 和 data_augmented 都已经有了 `tokens` 列
all_tokens = list(data['tokens']) + list(data_aug['tokens'])

# 构建词表
vocab = Counter(word for tokens in all_tokens for word in tokens)

# 建立 word -> index 映射
word_to_idx = {word: idx + 1 for idx, (word, _) in enumerate(vocab.most_common())}
vocab_size = len(word_to_idx) + 1  # +1 for padding index 0

data['numerical'] = data['tokens'].apply(lambda x: [word_to_idx.get(word, 0) for word in x])
data_aug['numerical'] = data_aug['tokens'].apply(lambda x: [word_to_idx.get(word, 0) for word in x])
vocab_size

26504

In [23]:
# 填充序列
max_len = 200
def truncate_and_pad(sequences, max_len):
    padded_sequences = []
    for seq in sequences:
        if len(seq) > max_len:  # 裁剪
            padded_sequences.append(seq[:max_len])
        else:  # 填充
            padded_sequences.append(seq + [0] * (max_len - len(seq)))
    return padded_sequences

X = truncate_and_pad(data['numerical'].values, max_len)
y = data['label'].values

# 数据划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=100)

# 转为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.long)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# 保存数据处理后的数据
torch.save({
    'train_loader': train_loader,
    'test_loader': test_loader,
    'vocab_size': vocab_size,
    'word_to_idx': word_to_idx,  # 保存词到索引的映射
}, './processed_data.pt')

In [24]:
# 填充序列
max_len = 200
def truncate_and_pad(sequences, max_len):
    padded_sequences = []
    for seq in sequences:
        if len(seq) > max_len:  # 裁剪
            padded_sequences.append(seq[:max_len])
        else:  # 填充
            padded_sequences.append(seq + [0] * (max_len - len(seq)))
    return padded_sequences

X = truncate_and_pad(data_aug['numerical'].values, max_len)
y = data_aug['label'].values

# 数据划分
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.1, random_state=100)

# 转为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.long)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# 保存数据处理后的数据
torch.save({
    'train_loader_aug': train_loader,
    'test_loader_aug': test_loader,
    'vocab_size_aug': vocab_size,
    'word_to_idx_aug': word_to_idx,  # 保存词到索引的映射
}, './processed_data_aug.pt')

## 原始lstm

In [47]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim


# 加载数据处理后的数据
data = torch.load('./processed_data.pt')
train_loader = data['train_loader']
test_loader = data['test_loader']
vocab_size = data['vocab_size']#每个文本的单词数量（固定好了），每个单词对应一个索引

# 定义 LSTM 模型
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout_rate):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)#输入索引，输出向量，每个索引对应一个embedding_dim大小的向量
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # 取最后时间步的输出
        x = self.fc(self.dropout(lstm_out))
        return x

# 模型参数
embedding_dim = 128
hidden_dim = 128
output_dim = 2
dropout_rate = 0

model = LSTMClassifier(vocab_size + 1, embedding_dim, hidden_dim, output_dim, dropout_rate)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 模型训练
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

    train_accuracy = correct / total
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}, Accuracy: {train_accuracy:.4f}")


# 保存模型
torch.save(model.state_dict(), './lstm_model_orig.pth')



C:\Users\ASUS\AppData\Local\Temp\ipykernel_21612\2319843758.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('./processed_data.pt')


Epoch 1/5, Loss: 55.7771, Accuracy: 0.7675
Epoch 2/5, Loss: 31.6702, Accuracy: 0.8739
Epoch 3/5, Loss: 27.4782, Accuracy: 0.8928
Epoch 4/5, Loss: 31.0279, Accuracy: 0.8917
Epoch 5/5, Loss: 43.9367, Accuracy: 0.8500


## 增强LSTM

In [48]:
# 加载数据处理后的数据
data = torch.load('./processed_data_aug.pt')
train_loader = data['train_loader_aug']
test_loader = data['test_loader_aug']
vocab_size = data['vocab_size_aug']#每个文本的单词数量（固定好了），每个单词对应一个索引

# 定义 LSTM 模型
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout_rate):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)#输入索引，输出向量，每个索引对应一个embedding_dim大小的向量
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # 取最后时间步的输出
        x = self.fc(self.dropout(lstm_out))
        return x

# 模型参数
embedding_dim = 128
hidden_dim = 128
output_dim = 2
dropout_rate = 0

model = LSTMClassifier(vocab_size + 1, embedding_dim, hidden_dim, output_dim, dropout_rate)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 模型训练
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

    train_accuracy = correct / total
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}, Accuracy: {train_accuracy:.4f}")


# 保存模型
torch.save(model.state_dict(), './lstm_model_aug.pth')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21612\2915289651.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('./processed_data_aug.pt')


Epoch 1/5, Loss: 56.7731, Accuracy: 0.7900
Epoch 2/5, Loss: 22.7473, Accuracy: 0.9419
Epoch 3/5, Loss: 21.2092, Accuracy: 0.9469
Epoch 4/5, Loss: 21.6655, Accuracy: 0.9422
Epoch 5/5, Loss: 47.1287, Accuracy: 0.8078


## 测试

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 模型结构（和之前一致）
# 定义 LSTM 模型
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout_rate):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)#输入索引，输出向量，每个索引对应一个embedding_dim大小的向量
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # 取最后时间步的输出
        x = self.fc(self.dropout(lstm_out))
        return x


# 加载处理数据
data_orig = torch.load('./processed_data.pt')
data_aug = torch.load('./processed_data_aug.pt')

vocab_size_orig = data_orig['vocab_size']
vocab_size_aug = data_aug['vocab_size_aug']
test_loader = data_orig['test_loader']  # 使用原始测试集

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# 加载模型函数
def load_model(path, vocab_size):
    model = LSTMClassifier(vocab_size + 1, 128, 128, 2, 0.3)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    model.eval()
    return model

model_orig = load_model('./lstm_model_orig.pth', vocab_size_orig)
model_aug = load_model('./lstm_model_aug.pth', vocab_size_aug)

# 融合预测函数
def evaluate_weighted_ensemble(model1, model2, test_loader, alpha):
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            out1 = F.softmax(model1(X_batch), dim=1)
            out2 = F.softmax(model2(X_batch), dim=1)

            fused = alpha * out1 + (1 - alpha) * out2
            _, predicted = torch.max(fused, 1)

            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

    return correct / total


# 输出准确率
acc_fused = evaluate_weighted_ensemble(model_orig, model_aug, test_loader, alpha=0.5)

print(f"加权融合准确率：{acc_fused:.4f}")


C:\Users\ASUS\AppData\Local\Temp\ipykernel_21612\1958019421.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_orig = torch.load('./processed_data.pt')
C:\Users\ASUS\

加权融合准确率：0.8250
